In [ ]:
import tensorflow as tf
print(tf.__version__)

In [30]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
import joblib # To load or export models
import os

In [31]:
# Load the data
data = pd.read_csv("royal_city_daily_avg.csv")  # Daily averages
# data = pd.read_csv("royal_city_late_morning_avg")  # Late morning averages (9:00AM - 11:00PM)

# Split data into features and target variables
X = data[['wind_speed', 'solar_radiation', 'relative_humidity', 'air_temp', 'canopy_temp_mean', 'soil_temp']]
y_water_content = data['water_content_mean']
y_water_potential = data['water_potential_mean']

# Normalize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and testing sets
X_train_water_content, X_test_water_content, y_train_water_content, y_test_water_content = train_test_split(X_scaled, y_water_content, test_size=0.2, random_state=42)
X_train_water_potential, X_test_water_potential, y_train_water_potential, y_test_water_potential = train_test_split(X_scaled, y_water_potential, test_size=0.2, random_state=42)

In [ ]:
# Create a neural network model for water content prediction
model_nn_water_content = Sequential([
    Dense(64, activation='relu', input_dim=X.shape[1]),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')
])

model_nn_water_content.summary()

In [ ]:
model_nn_water_content.compile(optimizer='adam', loss='mean_squared_error')
model_nn_water_content.fit(X_train_water_content, y_train_water_content, epochs=100, batch_size=32, validation_split=0.2)

In [ ]:
# Evaluate the model
results_water_content = {}

# (gives error)
# mse_nn_water_content, r2_nn_water_content = model_nn_water_content.evaluate(X_test_water_content, y_test_water_content)
# results_water_content['Neural Network'] = {'MSE': mse_nn_water_content, 'R2': r2_nn_water_content}

# Calculate Metrics
y_pred_water_content = model_nn_water_content.predict(X_test_water_content)
r2_nn_water_content = metrics.r2_score(y_test_water_content, y_pred_water_content)
mse_nn_water_content = metrics.mean_squared_error(y_test_water_content, y_pred_water_content)

# Update results dictionary
results_water_content['Neural Network'] = {'MSE': mse_nn_water_content, 'R2': r2_nn_water_content, 'predicted_values': y_pred_water_content}

In [ ]:
# Create a neural network model for water potential prediction
model_nn_water_potential = Sequential([
    Dense(64, activation='relu', input_dim=X.shape[1]),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')
])

model_nn_water_potential.summary()

In [ ]:

model_nn_water_potential.compile(optimizer='adam', loss='mean_squared_error')
model_nn_water_potential.fit(X_train_water_potential, y_train_water_potential, epochs=100, batch_size=32, validation_split=0.2)

In [ ]:
# Evaluate the model
results_water_potential = {}

# Calculate Metrics
y_pred_water_potential = model_nn_water_potential.predict(X_test_water_potential)
r2_nn_water_potential = metrics.r2_score(y_test_water_potential, y_pred_water_potential)
mse_nn_water_potential = metrics.mean_squared_error(y_test_water_potential, y_pred_water_potential)

# Update results dictionary
results_water_potential['Neural Network'] = {'MSE': mse_nn_water_potential, 'R2': r2_nn_water_potential, 'predicted_values': y_pred_water_potential}

In [ ]:
# Print results
print("Water Content Prediction Results:")
for model, results in results_water_content.items():
    print(f"{model}: MSE={results['MSE']:.4f}, R2={results['R2']:.4f}")

print("\nWater Potential Prediction Results:")
for model, results in results_water_potential.items():
    print(f"{model}: MSE={results['MSE']:.4f}, R2={results['R2']:.4f}")

In [ ]:
# Visualize results for water content and water potential predictions (scatter plots of predicted vs. actual values)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))  # Adjust figure size for square plots

# Water Content Prediction
ax1.scatter(y_test_water_content, y_pred_water_content, label=model)
ax1.set_xlabel("Actual Water Content")
ax1.set_ylabel("Predicted Water Content")
ax1.set_title("Predicted vs. Actual Water Content")
ax1.legend()

# Water Potential Prediction
ax2.scatter(y_test_water_potential, y_pred_water_potential, label=model)
ax2.set_xlabel("Actual Water Potential")
ax2.set_ylabel("Predicted Water Potential")
ax2.set_title("Predicted vs. Actual Water Potential")
ax2.legend()

plt.tight_layout()
plt.show()

In [40]:
# # Create the models folder if it doesn't exist
# model_folder = "models"
# if not os.path.exists(model_folder):
#     os.makedirs(model_folder)

# # Save the models (HDF5 format)
# model_nn_water_content.save(os.path.join(model_folder, "model_nn_water_content.h5"))
# model_nn_water_potential.save(os.path.join(model_folder, "model_nn_water_potential.h5")) 

In [41]:
# # Load the created neural network model
# model_nn_water_content = tf.keras.models.load_model(os.path.join(model_folder, "model_nn_water_content.h5"))
# model_nn_water_potential = tf.keras.models.load_model(os.path.join(model_folder, "model_nn_water_potential.h5"))